In [ ]:
# initial supervised data from: http://alt.qcri.org/semeval2015/task12/

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

def getDF(path):
    tree = ET.parse(path)
    root = tree.getroot()
    df = pd.DataFrame(columns=('rid','sid', 'text', 'category_polarity_list'))
    reviews = root.getchildren()
    rnum = len(reviews)
    j=0
    k=0
    for j in range(0,rnum):
        review = reviews[j]
        rid = review.attrib['rid']
        sentences = review.getchildren()[0]
        snum = len(sentences)
        for i in range(0,snum):
            sentence = sentences[i]
            sid = sentence.attrib['id']
            text = sentence[0].text
            opinions=[]
            if len(sentence)>1:
                opinions = sentence[1]
            category_polarity_list = []
            for opinion in opinions:
                category = opinion.attrib['category']
                polarity = opinion.attrib['polarity']
                category_polarity_list.append((category, polarity))
            row = dict(zip(['rid','sid', 'text', 'category_polarity_list'], [rid, sid, text, category_polarity_list]))
            row_s = pd.Series(row)
            row_s.name = k
            df = df.append(row_s)
            k +=1

    getCategorys = lambda lst: list(map(lambda x: x[0], lst))
    df['categorys']  = df['category_polarity_list'].apply(getCategorys)
    
    getCategoryNum = lambda lst: len(lst)
    df['category_num']  = df['categorys'].apply(getCategoryNum)
    return df

In [56]:
trainDF = getDF('ABSA-15_Laptops_Train_Data.xml')
testDF = getDF('ABSA15_Laptops_Test.xml')

In [57]:
trainDF.head(10)

,rid,sid,text,category_polarity_list,categorys,category_num
0,79,79:0,Being a PC user my whole life....,[],[],0
1,79,79:1,This computer is absolutely AMAZING!!!,"[(LAPTOP#GENERAL, positive)]",[LAPTOP#GENERAL],1
2,79,79:2,10 plus hours of battery...,"[(BATTERY#OPERATION_PERFORMANCE, positive)]",[BATTERY#OPERATION_PERFORMANCE],1
3,79,79:3,super fast processor and really nice graphics ...,"[(CPU#OPERATION_PERFORMANCE, positive), (GRAPH...","[CPU#OPERATION_PERFORMANCE, GRAPHICS#GENERAL]",2
4,79,79:4,and plenty of storage with 250 gb(though I wil...,"[(HARD_DISC#DESIGN_FEATURES, positive)]",[HARD_DISC#DESIGN_FEATURES],1
5,79,79:5,This computer is really fast and I'm shocked a...,"[(LAPTOP#OPERATION_PERFORMANCE, positive), (LA...","[LAPTOP#OPERATION_PERFORMANCE, LAPTOP#USABILITY]",2
6,79,79:6,I've only had mine a day but I'm already used ...,"[(LAPTOP#USABILITY, positive)]",[LAPTOP#USABILITY],1
7,79,79:7,MACS ARE AMAZING!!!,[],[],0
8,79,79:8,GET THIS COMPUTER FOR PORTABILITY AND FAST PRO...,"[(LAPTOP#PORTABILITY, positive), (CPU#OPERATIO...","[LAPTOP#PORTABILITY, CPU#OPERATION_PERFORMANCE]",2
9,10,10:0,the laptop was really good and it goes really ...,"[(LAPTOP#GENERAL, positive), (LAPTOP#OPERATION...","[LAPTOP#GENERAL, LAPTOP#OPERATION_PERFORMANCE]",2


In [58]:
trainDF['category_num'].max()

5

In [51]:
trainDF['category_num'].argmax()

791

In [59]:
trainDF.ix[791].text

'I actually had the hard drive replaced twice, the mother board once, the dvd drive twice, then they FINALLY agreed to replace it, (ALL OF THIS IN LESS THAN 1 1/2 YEARS!)only to get another "HP/Compaq" piece of crap.'

In [60]:
trainDF.ix[791].category_polarity_list

[('HARD_DISC#QUALITY', 'negative'),
 ('MOTHERBOARD#QUALITY', 'negative'),
 ('OPTICAL_DRIVES#QUALITY', 'negative'),
 ('SUPPORT#QUALITY', 'negative'),
 ('LAPTOP#QUALITY', 'negative')]

In [61]:
trainDF.count()

rid                       1739
sid                       1739
text                      1739
category_polarity_list    1739
categorys                 1739
category_num              1739
dtype: int64

In [63]:
testDF.count()

rid                       761
sid                       761
text                      761
category_polarity_list    761
categorys                 761
category_num              761
dtype: int64

In [64]:
trainDF.to_csv('sentenceTrainDF.csv')

In [65]:
testDF.to_csv('sentenceTestDF.csv')